The purpose of this notebook is to explore the raw 311 Service Requests dataset in order to understand volumes, distributions, and processing characteristics, and to define a realistic analytical scope that will later be implemented in Excel using Power Query.# 311 Service Requests – EDA for Processing Performance

This notebook performs an exploratory data analysis (EDA) on a scoped subset of the NYC 311 Service Requests dataset.

The objective is **not** to build a production-ready dataset, but to:
- understand volumes and processing times,
- validate analytical assumptions,
- define realistic KPIs and SLA thresholds,
- decide the scope and logic to be implemented later in Excel (Power Query).

All final data preparation and reporting will be performed in Excel.


In [22]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", "{:.3f}".format)



In [23]:
PATH = "../data/raw/311_Service_Requests_from_2020_to_Present_20260204.csv"

df = pd.read_csv(
    PATH,
    sep=";",          # Socrata export uses semicolon separator
    low_memory=False
)

df.shape






(245049, 44)

## Dataset Overview

- Source: NYC OpenData – 311 Service Requests
- Scope applied at source:
  - Agency: NYPD
  - Request type: Noise – Residential
  - Period: August 2025 → February 2026
- Volume: ~245k requests

This scope ensures homogeneous processing logic and realistic operational monitoring.


In [24]:
df.columns.tolist()


['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Problem (formerly Complaint Type)',
 'Problem Detail (formerly Descriptor)',
 'Additional Details',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'Council District',
 'Police Precinct',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location']

## Relevant Columns for Operational Analysis

Based on the dataset structure, the following columns are relevant for processing performance monitoring:

- Created Date: request creation timestamp
- Closed Date: request resolution timestamp
- Agency / Agency Name: responsible service
- Problem (formerly Complaint Type): request category
- Problem Detail (formerly Descriptor): request sub-type
- Status: request status (Closed / In Progress)
- Borough: geographic dimension

Other fields are intentionally ignored at this stage.


In [25]:
df[["Agency", "Problem (formerly Complaint Type)"]].value_counts()


Agency  Problem (formerly Complaint Type)
NYPD    Noise - Residential                  245049
Name: count, dtype: int64

In [26]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("(", "", regex=False)
      .str.replace(")", "", regex=False)
)

df.columns.tolist()



['unique_key',
 'created_date',
 'closed_date',
 'agency',
 'agency_name',
 'problem_formerly_complaint_type',
 'problem_detail_formerly_descriptor',
 'additional_details',
 'location_type',
 'incident_zip',
 'incident_address',
 'street_name',
 'cross_street_1',
 'cross_street_2',
 'intersection_street_1',
 'intersection_street_2',
 'address_type',
 'city',
 'landmark',
 'facility_type',
 'status',
 'due_date',
 'resolution_description',
 'resolution_action_updated_date',
 'community_board',
 'council_district',
 'police_precinct',
 'bbl',
 'borough',
 'x_coordinate_state_plane',
 'y_coordinate_state_plane',
 'open_data_channel_type',
 'park_facility_name',
 'park_borough',
 'vehicle_type',
 'taxi_company_borough',
 'taxi_pick_up_location',
 'bridge_highway_name',
 'bridge_highway_direction',
 'road_ramp',
 'bridge_highway_segment',
 'latitude',
 'longitude',
 'location']

In [27]:
df["created_date"] = pd.to_datetime(df["created_date"], errors="coerce")
df["closed_date"] = pd.to_datetime(df["closed_date"], errors="coerce")

df["created_date"].min(), df["created_date"].max()



(Timestamp('2025-08-04 09:22:41'), Timestamp('2026-02-03 02:04:35'))

## Request Status Distribution

Understanding the share of closed vs open requests is essential to assess backlog behavior
and determine whether the analysis should focus on stock accumulation or processing time.


In [28]:
df["status"].value_counts(normalize=True).round(3)


status
Closed        0.999
In Progress   0.001
Unspecified   0.000
Name: proportion, dtype: float64

## Processing Time Calculation

Initial tests using day-based metrics showed that most requests are resolved within the same day.
As a result, **hour-based metrics** are more relevant than day-based ones for this use case.


In [29]:
df["lead_time_hours"] = (
    df["closed_date"] - df["created_date"]
).dt.total_seconds() / 3600


In [30]:
df_closed = df[df["status"] == "Closed"]
df_closed.shape



(244824, 45)

In [31]:
df_closed["lead_time_hours"].describe(
    percentiles=[0.5, 0.75, 0.9, 0.95]
)


count   244824.000
mean         2.152
std          3.346
min         -0.881
50%          1.042
75%          2.483
90%          5.305
95%          8.264
max        186.271
Name: lead_time_hours, dtype: float64

## Interpretation of Processing Times

- Most requests are resolved within a few hours.
- The distribution is strongly right-skewed.
- Median and percentile-based indicators are more meaningful than averages.
- Performance issues are concentrated in a limited number of delayed cases.


## SLA Definition Decision

Based on observed processing times, a **4-hour SLA** is retained for operational monitoring.

Rationale:
- Close to the 75th percentile
- Strict enough to highlight delays
- Realistic given observed performance
- Easy to interpret for non-technical stakeholders

This SLA will be implemented in Excel using Power Query.


## EDA Conclusions and Next Steps

This exploratory analysis confirms that:
- The analytical scope is consistent and homogeneous.
- Processing performance is generally strong, with a limited number of delayed cases.
- Hour-based KPIs are required to capture meaningful variations.
- A 4-hour SLA is an appropriate operational threshold.

The next steps are:
- Implement all filtering and calculations in Excel using Power Query.
- Build operational KPIs and dashboards for performance monitoring.
- Focus reporting on delays, trends, and SLA compliance.

No further data transformation will be performed in Python.
